# The Viewer

In [ ]:
import numpy as np
import json
import time
import ipywidgets as widgets
from cad_viewer_widget import CadViewer, AnimationTrack
from sidecar import Sidecar

names = ["hexapod", "hexapod_m", "box", "box1", "boxes", "edges", "faces", "vertices"]
objects = {}
states = {}

for name in names:
    with open(f"examples/{name}.json", "r") as fd:
        objects[name] = json.load(fd)

    with open(f"examples/{name}-states.json", "r") as fd:
        states[name] = json.load(fd)

## Open viewer in cell

In [ ]:
cv2 = CadViewer(
    cad_width=640, 
    height=480, 
    tools=True,
)

In [ ]:
cv2.add_shapes(
        objects["hexapod"],
        states["hexapod"],
        ortho=False,
        control="orbit",
        grid=(True, False, True)
)

In [ ]:
from ipywidgets.embed import embed_minimal_html
# !!! sc.close()
embed_minimal_html('export.html', views=[cv2.widget], title='Widgets export')

In [ ]:
sc = Sidecar(title="Tests")

with sc:
    cv = CadViewer(
        cad_width=640, 
        height=480, 
        tree_width=250, 
        theme="light", 
        tools=True
    )

## Observe Javascript events

In [ ]:
cv.js_debug = True

## Adding shapes

In [ ]:
def show(name):
    cv.add_shapes(
        objects[name],
        states[name],
        ortho=True,
        control="trackball",
        ticks=20,
        axes=True,
        axes0=False,
        grid=[True, False, False], 
        transparent=False,
        black_edges=False,
        edge_color="#707070",
        ambient_intensity=0.5,
        direct_intensity=0.3,
        reset_camera=True,
    #    position=[-580.0, 200.0, 750.0],
    #    quaternion=[-0.33, 0.026, 0.91, 0.24],
    #    zoom=0.5,
    #    timeit=True,
    #    animation_loop=False
    )

menu = widgets.Dropdown(
    options=names,
    value=names[0],
    description='Number:',
    disabled=False,
)

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        show(change['new'])

menu.observe(on_change)

show(names[0])
menu

# Camera position control

In [ ]:
p0 = cv.position
if cv.control == "trackball":
    q0 = cv.quaternion
else:
    q0 = None
p0, q0

In [ ]:
cv.position = [-580.0, 200.0, 750.0]

if cv.control == "trackball":
    cv.quaternion = [-0.33, 0.026, 0.91, 0.24]

cv.zoom = 0.5

In [ ]:
cv.position = p0

if cv.control == "trackball":
    cv.quaternion = q0
    
cv.zoom = 1

# Widget interaction

In [ ]:
cv.update_states({
    '/bottom/bottom_0': (1,0),
    '/bottom/top/top_0': [0,1],
})

In [ ]:
cv.update_states({
    '/bottom/bottom_0': (1,1),
    '/bottom/top/top_0': [1,1],
})

In [ ]:
cv.ambient_intensity = 0.9
cv.direct_intensity = 0.5

In [ ]:
cv.ambient_intensity = 0.5
cv.direct_intensity = 0.3

In [ ]:
ec = cv.edge_color

In [ ]:
cv.edge_color = "#ff0000"

In [ ]:
cv.edge_color = ec

In [ ]:
cv.grid = [not g for g in cv.widget.grid]

In [ ]:
cv.axes = not cv.axes
cv.axes0 = not cv.axes0
cv.transparent = not cv.transparent
cv.black_edges = not cv.black_edges

In [ ]:
cv.tools = not cv.tools

In [ ]:
cv.ortho = not cv.ortho

In [ ]:
cv.zoom_speed = 5
cv.pan_speed = 5
cv.rotate_speed = 5

In [ ]:
cv.zoom_speed =1
cv.pan_speed =1
cv.rotate_speed =1

In [ ]:
cv.last_pick

# Clipping handling

In [ ]:
cv.select_clipping()

In [ ]:
cv.clip_intersection = not cv.clip_intersection

In [ ]:
cv.clip_planes = not cv.clip_planes

In [ ]:
cv.clip_value_0 = 10
cv.clip_value_1 = -50
cv.clip_value_2 = 40

In [ ]:
cv.clip_normal_0

In [ ]:
cv.clip_value_2

In [ ]:
cv.clip_normal_0 = (-0.35, -0.35, -0.35)

In [ ]:
cv.clip_normal_0 = (-1, 0, 0)

In [ ]:
cv.select_tree()

# Rotations

In [ ]:
for i in range(10):
    cv.rotate_x(1)
    cv.rotate_y(3)
    cv.rotate_z(5)
    time.sleep(0.1)

In [ ]:
for i in range(10):
    cv.rotate_z(-5)
    cv.rotate_y(-3)
    cv.rotate_x(-1)
    time.sleep(0.1)    

In [ ]:
cv.rotate_up(30)
cv.rotate_left(30)

In [ ]:
cv.rotate_left(-30)
cv.rotate_up(-30)

# Animation

In [ ]:
cv = cv2

In [ ]:
import numpy as np
horizontal_angle = 25

leg_names = {
    "right_back", "right_middle", "right_front", 
    "left_back",  "left_middle", "left_front",
    
}

def intervals(count):
    r = [ min(180, (90 + i*(360 // count)) % 360) for i in range(count)]
    return r 

def times(end, count):
    return np.linspace(0, end, count+1).tolist()
    
def vertical(count, end, offset, reverse):
    ints = intervals(count)
    heights = [round(35 * np.sin(np.deg2rad(x)) - 15, 1) for x in ints]
    heights.append(heights[0])
    return times(end, count), heights[offset:] + heights[1:offset+1]

def horizontal(end, reverse):
    factor = 1 if reverse else -1
    return times(end, 4), [0, factor * horizontal_angle, 0, -factor * horizontal_angle, 0]

leg_group = ("left_front", "right_middle", "left_back")

In [ ]:
tracks = []

for name in leg_names:
    # move upper leg
    cv.add_track(AnimationTrack(f"/bottom/{name}", "rz", *horizontal(4, "middle" in name)))

cv.animate(5)
cv.play()

In [ ]:
cv.stop()

In [ ]:
for name in leg_names:
    # move lower leg
    cv.add_track(AnimationTrack(f"/bottom/{name}/lower", "rz", *vertical(8, 4, 0 if name in leg_group else 4, "left" in name)))
    
cv.animate(2)
cv.play()

In [ ]:
cv.clear_tracks()

# No Widget, Javascript only

In [ ]:
import json
from IPython.display import Javascript, HTML
from cad_viewer_widget.utils import serializer

In [ ]:
display(Javascript("""
var script = document.createElement('script');
script.type = 'text/javascript';
script.src = 'http://banach:8888/lab/extensions/cad-viewer-widget/static/lib_index_js.b616f2d187f2f136095a.js';
document.head.appendChild(script);
"""))

In [ ]:
shapes = json.dumps(objects["box1"], default=serializer)
states2 = json.dumps(states["box1"])

display(HTML("<div id='cad_view'></div>"))

display(Javascript(f"""
const options = {{
    theme: "light",
    ortho: true,
    control: "trackball",
    // control: "orbit",
    normalLen: 0,
    cadWidth: 800,
    height: 600,
    treeWidth: 240,
    ticks: 10,
    normalLen: 0,
    ambientIntensity: 0.9,
    directIntensity: 0.12,
    transparent: false,
    blackEdges: false,
    axes: true,
    grid: [false, false, false], 
    timeit: false,
    rotateSpeed: 1
}};

const container = document.getElementById("cad_view");
const display = new Display(container, options);
display.setAnimationControl(false)
const shapes = JSON.parse('{shapes}');
const states = JSON.parse('{states2}');
console.log(shapes)
console.log(states)
"""))


In [ ]:
sc.close()

In [ ]:
print("[[\"/bottom/right_front\", \"rz\", [0.0, 1.0, 2.0, 3.0, 4.0], [0, -25, 0, 25, 0]], [\"/bottom/left_middle\", \"rz\", [0.0, 1.0, 2.0, 3.0, 4.0], [0, 25, 0, -25, 0]], [\"/bottom/left_front\", \"rz\", [0.0, 1.0, 2.0, 3.0, 4.0], [0, -25, 0, 25, 0]], [\"/bottom/right_middle\", \"rz\", [0.0, 1.0, 2.0, 3.0, 4.0], [0, 25, 0, -25, 0]], [\"/bottom/left_back\", \"rz\", [0.0, 1.0, 2.0, 3.0, 4.0], [0, -25, 0, 25, 0]], [\"/bottom/right_back\", \"rz\", [0.0, 1.0, 2.0, 3.0, 4.0], [0, -25, 0, 25, 0]], [\"/bottom/right_front/lower\", \"rz\", [0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0], [-15.0, -15.0, -15.0, 9.7, 20.0, 9.7, -15.0, -15.0, -15.0]], [\"/bottom/left_middle/lower\", \"rz\", [0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0], [-15.0, -15.0, -15.0, 9.7, 20.0, 9.7, -15.0, -15.0, -15.0]], [\"/bottom/left_front/lower\", \"rz\", [0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0], [20.0, 9.7, -15.0, -15.0, -15.0, -15.0, -15.0, 9.7, 20.0]], [\"/bottom/right_middle/lower\", \"rz\", [0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0], [20.0, 9.7, -15.0, -15.0, -15.0, -15.0, -15.0, 9.7, 20.0]], [\"/bottom/left_back/lower\", \"rz\", [0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0], [20.0, 9.7, -15.0, -15.0, -15.0, -15.0, -15.0, 9.7, 20.0]], [\"/bottom/right_back/lower\", \"rz\", [0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0], [-15.0, -15.0, -15.0, 9.7, 20.0, 9.7, -15.0, -15.0, -15.0]]]")

In [ ]:
0xe8/255, 0xb0/255, 0x24/255